In [1]:
import os
# Find the latest version of spark 3.0  from http://www-us.apache.org/dist/spark/ and enter as the spark version
# For example:
# spark_version = 'spark-3.0.1'
spark_version = 'spark-3.0.1'
os.environ['SPARK_VERSION']=spark_version

# Install Spark and Java
!apt-get update
!apt-get install openjdk-11-jdk-headless -qq > /dev/null
!wget -q http://www-us.apache.org/dist/spark/$SPARK_VERSION/$SPARK_VERSION-bin-hadoop2.7.tgz
!tar xf $SPARK_VERSION-bin-hadoop2.7.tgz
!pip install -q findspark

# Set Environment Variables
import os
os.environ["JAVA_HOME"] = "/usr/lib/jvm/java-11-openjdk-amd64"
os.environ["SPARK_HOME"] = f"/content/{spark_version}-bin-hadoop2.7"

# Start a SparkSession
import findspark
findspark.init()

Hit:1 https://cloud.r-project.org/bin/linux/ubuntu bionic-cran40/ InRelease
Ign:2 https://developer.download.nvidia.com/compute/cuda/repos/ubuntu1804/x86_64  InRelease
Ign:3 https://developer.download.nvidia.com/compute/machine-learning/repos/ubuntu1804/x86_64  InRelease
Hit:4 https://developer.download.nvidia.com/compute/cuda/repos/ubuntu1804/x86_64  Release
Hit:5 https://developer.download.nvidia.com/compute/machine-learning/repos/ubuntu1804/x86_64  Release
Hit:6 http://ppa.launchpad.net/c2d4u.team/c2d4u4.0+/ubuntu bionic InRelease
Get:7 http://security.ubuntu.com/ubuntu bionic-security InRelease [88.7 kB]
Hit:9 http://archive.ubuntu.com/ubuntu bionic InRelease
Hit:11 http://ppa.launchpad.net/graphics-drivers/ppa/ubuntu bionic InRelease
Get:12 http://archive.ubuntu.com/ubuntu bionic-updates InRelease [88.7 kB]
Get:13 http://archive.ubuntu.com/ubuntu bionic-backports InRelease [74.6 kB]
Fetched 252 kB in 2s (114 kB/s)
Reading package lists... Done


In [2]:
!wget https://jdbc.postgresql.org/download/postgresql-42.2.16.jar

--2020-10-18 17:39:58--  https://jdbc.postgresql.org/download/postgresql-42.2.16.jar
Resolving jdbc.postgresql.org (jdbc.postgresql.org)... 72.32.157.228, 2001:4800:3e1:1::228
Connecting to jdbc.postgresql.org (jdbc.postgresql.org)|72.32.157.228|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 1002883 (979K) [application/java-archive]
Saving to: ‘postgresql-42.2.16.jar.1’

postgresql-42.2.16. 100%[===================>] 979.38K  1009KB/s    in 1.0s    

2020-10-18 17:40:00 (1009 KB/s) - ‘postgresql-42.2.16.jar.1’ saved [1002883/1002883]



In [3]:
from pyspark.sql import SparkSession
spark = SparkSession.builder.appName("Wine").config("spark.driver.extraClassPath","/content/postgresql-42.2.16.jar").getOrCreate()

In [4]:
# Read in data from S3 Buckets
from pyspark import SparkFiles
url ="https://wine5-bucket.s3.us-east-2.amazonaws.com/Red_Wine_USA.csv"
spark.sparkContext.addFile(url)
red_wine_df = spark.read.csv(SparkFiles.get("Red_Wine_USA.csv"), sep=",", header=True)

#Show DataFrame
red_wine_df.show(10)

+---+--------------------+-------+--------------------+-----+----------+-------+-------+-----------+-----+----------------+----------------+
|_c0|                wine|wine_id|         appellation|color|   regions|country|vintage|is_primeurs|score|confidence_index|journalist_count|
+---+--------------------+-------+--------------------+-----+----------+-------+-------+-----------+-----+----------------+----------------+
|  0|Ridge Vineyards, ...| 120786|Santa Cruz Mountains|  Red|California|    Usa|   2013|      False|95.83|              C+|               8|
|  1|Quintessa, Ruther...| 118360|          Rutherford|  Red|California|    Usa|   2015|      False|95.77|               C|               3|
|  2|Harlan Estate, Na...|  81980|         Napa Valley|  Red|California|    Usa|   2001|      False|98.89|              A+|               6|
|  3|Dalla Valle Viney...|  48290|         Napa Valley|  Red|California|    Usa|   2001|      False|96.49|              B+|               5|
|  4|Philip T

In [5]:
# Read in data from S3 Buckets
from pyspark import SparkFiles
url ="https://wine5-bucket.s3.us-east-2.amazonaws.com/White_Wine_USA.csv"
spark.sparkContext.addFile(url)
white_wine_df = spark.read.csv(SparkFiles.get("White_Wine_USA.csv"), sep=",", header=True)

#Show DataFrame
white_wine_df.show(10)

+---+--------------------+-------+--------------------+-----+----------+-------+-------+-----------+-----+----------------+----------------+
|_c0|                wine|wine_id|         appellation|color|   regions|country|vintage|is_primeurs|score|confidence_index|journalist_count|
+---+--------------------+-------+--------------------+-----+----------+-------+-------+-----------+-----+----------------+----------------+
|  0|Ridge Vineyards, ...| 120785|Santa Cruz Mountains|White|California|    Usa|   2016|      False|94.39|               B|               3|
|  1|Stags' Leap Winer...| 129823|         Napa Valley|White|California|    Usa|   2016|      False|89.83|              A+|               3|
|  2|Joseph Phelps Vin...|  89569|        Sonoma Coast|White|California|    Usa|   2016|      False|90.73|              C+|               4|
|  3|Crossbarn By Paul...|  47377|        Sonoma Coast|White|California|    Usa|   2016|      False|90.39|              B+|               3|
|  4|Mount Ed

In [6]:
red_wine_df= red_wine_df.drop('wine_type', 'classification')
red_wine_df.show(10)

+---+--------------------+-------+--------------------+-----+----------+-------+-------+-----------+-----+----------------+----------------+
|_c0|                wine|wine_id|         appellation|color|   regions|country|vintage|is_primeurs|score|confidence_index|journalist_count|
+---+--------------------+-------+--------------------+-----+----------+-------+-------+-----------+-----+----------------+----------------+
|  0|Ridge Vineyards, ...| 120786|Santa Cruz Mountains|  Red|California|    Usa|   2013|      False|95.83|              C+|               8|
|  1|Quintessa, Ruther...| 118360|          Rutherford|  Red|California|    Usa|   2015|      False|95.77|               C|               3|
|  2|Harlan Estate, Na...|  81980|         Napa Valley|  Red|California|    Usa|   2001|      False|98.89|              A+|               6|
|  3|Dalla Valle Viney...|  48290|         Napa Valley|  Red|California|    Usa|   2001|      False|96.49|              B+|               5|
|  4|Philip T

In [7]:
red_wine_df.count()

4364

In [8]:
white_wine_df.count()

784

In [9]:
# Create Red Wine Table 
red_wine_df = red_wine_df.select(["appellation", "regions", "vintage", "score", "confidence_index", "journalist_count"])
red_wine_df.show(10)

+--------------------+----------+-------+-----+----------------+----------------+
|         appellation|   regions|vintage|score|confidence_index|journalist_count|
+--------------------+----------+-------+-----+----------------+----------------+
|Santa Cruz Mountains|California|   2013|95.83|              C+|               8|
|          Rutherford|California|   2015|95.77|               C|               3|
|         Napa Valley|California|   2001|98.89|              A+|               6|
|         Napa Valley|California|   2001|96.49|              B+|               5|
|         Napa Valley|California|   2001| 96.2|               B|               3|
|         Napa Valley|California|   2001| 95.6|               B|               5|
|          Rutherford|California|   2001|95.19|              C+|               3|
| Stags Leap District|California|   2001|93.69|               C|               6|
|         Napa Valley|California|   2001|93.66|              B+|               5|
|         Napa V

In [10]:
#Drop null values
dropna_df = red_wine_df.dropna()
dropna_df.show()

+--------------------+----------+-------+-----+----------------+----------------+
|         appellation|   regions|vintage|score|confidence_index|journalist_count|
+--------------------+----------+-------+-----+----------------+----------------+
|Santa Cruz Mountains|California|   2013|95.83|              C+|               8|
|          Rutherford|California|   2015|95.77|               C|               3|
|         Napa Valley|California|   2001|98.89|              A+|               6|
|         Napa Valley|California|   2001|96.49|              B+|               5|
|         Napa Valley|California|   2001| 96.2|               B|               3|
|         Napa Valley|California|   2001| 95.6|               B|               5|
|          Rutherford|California|   2001|95.19|              C+|               3|
| Stags Leap District|California|   2001|93.69|               C|               6|
|         Napa Valley|California|   2001|93.66|              B+|               5|
|         Napa V

In [11]:
#Drop null values
white_dropna_df = white_wine_df.dropna()
dropna_df.show()

+--------------------+----------+-------+-----+----------------+----------------+
|         appellation|   regions|vintage|score|confidence_index|journalist_count|
+--------------------+----------+-------+-----+----------------+----------------+
|Santa Cruz Mountains|California|   2013|95.83|              C+|               8|
|          Rutherford|California|   2015|95.77|               C|               3|
|         Napa Valley|California|   2001|98.89|              A+|               6|
|         Napa Valley|California|   2001|96.49|              B+|               5|
|         Napa Valley|California|   2001| 96.2|               B|               3|
|         Napa Valley|California|   2001| 95.6|               B|               5|
|          Rutherford|California|   2001|95.19|              C+|               3|
| Stags Leap District|California|   2001|93.69|               C|               6|
|         Napa Valley|California|   2001|93.66|              B+|               5|
|         Napa V

In [12]:
# Load in a sql function to use columns
from pyspark.sql.functions import col

# Filter for only columns with journalist_count > 5
cleaned_df = dropna_df.filter(col("journalist_count") > 5)
cleaned_df.show()

+--------------------+----------+-------+-----+----------------+----------------+
|         appellation|   regions|vintage|score|confidence_index|journalist_count|
+--------------------+----------+-------+-----+----------------+----------------+
|Santa Cruz Mountains|California|   2013|95.83|              C+|               8|
|         Napa Valley|California|   2001|98.89|              A+|               6|
| Stags Leap District|California|   2001|93.69|               C|               6|
|         Napa Valley|California|   2001| 93.6|               B|              10|
|         Napa Valley|California|   2001| 92.2|              C+|               6|
|         Napa Valley|California|   2001|91.88|               B|               8|
|Santa Cruz Mountains|California|   1991| 95.9|               B|               8|
|         Napa Valley|California|   2010|98.41|               B|               8|
|         Napa Valley|California|   2010| 96.3|              C+|               9|
|           St H

In [13]:
cleaned_df.count()

446